In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests

In [2]:
wine_data = pd.read_csv('winemag-data-130k-v2.csv')

In [3]:
wine_data.shape

(129971, 14)

In [4]:
wine_data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [5]:
wine_data['country'].value_counts() #Italy is third for most value counts!

country
US                        54504
France                    22093
Italy                     19540
Spain                      6645
Portugal                   5691
Chile                      4472
Argentina                  3800
Austria                    3345
Australia                  2329
Germany                    2165
New Zealand                1419
South Africa               1401
Israel                      505
Greece                      466
Canada                      257
Hungary                     146
Bulgaria                    141
Romania                     120
Uruguay                     109
Turkey                       90
Slovenia                     87
Georgia                      86
England                      74
Croatia                      73
Mexico                       70
Moldova                      59
Brazil                       52
Lebanon                      35
Morocco                      28
Peru                         16
Ukraine                      14


In [6]:
# Sort the countries based on the count of entries in descending order
sorted_countries = wine_data['country'].value_counts().sort_values(ascending=False)

# Select the top 10 countries
top_10_countries = sorted_countries.head(10)

print(top_10_countries)

country
US           54504
France       22093
Italy        19540
Spain         6645
Portugal      5691
Chile         4472
Argentina     3800
Austria       3345
Australia     2329
Germany       2165
Name: count, dtype: int64


In [7]:
wine_data['province'].value_counts() #Top province is California

province
California        36247
Washington         8639
Bordeaux           5941
Tuscany            5897
Oregon             5373
                  ...  
Messinia              1
Markopoulo            1
Paardeberg            1
Krania Olympus        1
Neuchâtel             1
Name: count, Length: 425, dtype: int64

In [8]:
# Sort the province based on the count of entries in descending order
sorted_province = wine_data['province'].value_counts().sort_values(ascending=False)

# Select the top 10 province
top_10_province = sorted_province.head(10)

print(top_10_province)

province
California          36247
Washington           8639
Bordeaux             5941
Tuscany              5897
Oregon               5373
Burgundy             3980
Northern Spain       3851
Piedmont             3729
Mendoza Province     3264
Veneto               2716
Name: count, dtype: int64


In [9]:
#top province within the US
top_provinces_us = wine_data['province'][wine_data['country'] == 'US'].value_counts().sort_values(ascending=False).head(10)
print(top_provinces_us)

province
California    36247
Washington     8639
Oregon         5373
New York       2688
Virginia        777
Idaho           192
Michigan        114
America          95
Texas            94
Colorado         68
Name: count, dtype: int64


In [10]:
wine_data['variety'].value_counts() #Majority are Pinot Noir and Chardonnay (makes sense with Cali being 1)

variety
Pinot Noir                    13272
Chardonnay                    11753
Cabernet Sauvignon             9472
Red Blend                      8946
Bordeaux-style Red Blend       6915
                              ...  
Cabernet Sauvignon-Barbera        1
Sauvignonasse                     1
Forcallà                          1
Meseguera                         1
Bobal-Cabernet Sauvignon          1
Name: count, Length: 707, dtype: int64

In [11]:
# Sort the variety based on the count of entries in descending order
sorted_variety = wine_data['variety'].value_counts().sort_values(ascending=False)

# Select the top 10 variety
top_10_variety = sorted_variety.head(10)

print(top_10_variety)

variety
Pinot Noir                  13272
Chardonnay                  11753
Cabernet Sauvignon           9472
Red Blend                    8946
Bordeaux-style Red Blend     6915
Riesling                     5189
Sauvignon Blanc              4967
Syrah                        4142
Rosé                         3564
Merlot                       3102
Name: count, dtype: int64


In [12]:
wine_data['points'].value_counts() #Points range from 80 to 100

points
88     17207
87     16933
90     15410
86     12600
89     12226
91     11359
92      9613
85      9530
93      6489
84      6480
94      3758
83      3025
82      1836
95      1535
81       692
96       523
80       397
97       229
98        77
99        33
100       19
Name: count, dtype: int64

In [13]:
#THE VARIETY WITH THE HIGHEST POINT 100

#Sort the points in ascending order
sorted_counts = wine_data['points'].value_counts().sort_index(ascending=False)
#Top point 88
top_point = sorted_counts.index[0]

variety_for_top_point = wine_data[wine_data['points'] == top_point]['variety']#.iloc[0]
variety_for_top_point.value_counts()

variety
Bordeaux-style Red Blend      5
Champagne Blend               2
Syrah                         2
Muscat                        1
Prugnolo Gentile              1
Merlot                        1
Portuguese Red                1
Sangiovese                    1
Cabernet Sauvignon            1
Chardonnay                    1
Sangiovese Grosso             1
Port                          1
Bordeaux-style White Blend    1
Name: count, dtype: int64

In [14]:
#THE TOP 10 VARIETIES WITH THE HIGHEST value_count POINT 88

top_point = wine_data['points'].value_counts().idxmax()

variety_top_value_point = wine_data[wine_data['points'] == top_point]['variety']#.iloc[0]

variety_top_value_point.value_counts().head(10)

variety
Pinot Noir                  1434
Chardonnay                  1393
Red Blend                   1354
Cabernet Sauvignon          1064
Bordeaux-style Red Blend     849
Sauvignon Blanc              720
Riesling                     636
Rosé                         505
Syrah                        458
Sangiovese                   444
Name: count, dtype: int64

In [15]:
len(variety_top_value_point)

17207

In [16]:
wine_data['description'].nunique() #Filter out unique values by key words (fruity, acidity, citrus...)

119955

In [17]:
#wine_data['description'].value_counts() TOO MANY DESCRIPTIONS TO SUCCEFULY FILTER OUT

In [18]:
wine_data['variety'].nunique()

707

In [19]:
wine_data['country'].nunique()

43

In [20]:
wine_data.drop(columns=['Unnamed: 0', 'designation', 'region_1', 'region_2',
                        'taster_name', 'taster_twitter_handle'], inplace=True)

In [21]:
wine_data.dtypes

country         object
description     object
points           int64
price          float64
province        object
title           object
variety         object
winery          object
dtype: object

#### Categorical Data: 
Country, Description, Province, Title, Variety, Winery

#### Numerical Data: 
Points, Price

In [22]:
wine_data.isna().sum() #No Nan values in description, few in country, only 1 in variety

country          63
description       0
points            0
price          8996
province         63
title             0
variety           1
winery            0
dtype: int64

#### Regular expression pattern to match the whole word "__________"
pattern = r"\b(fruit)\b"

#### wine_data["description"] = wine_data['description'].fillna('N/A')

wine_data["description"] = wine_data["description"].astype(str)
wine_data["description"] = wine_data["description"].apply(lambda word : word.strip())

#### Filter rows containing the word "_________" (case-insensitive, if required)
fruit_wine = wine_data["description"].str.extract(pattern)

In [23]:
#Should we drop price?

------------------------------------------------------------------------------------------------------------

In [24]:
vivino_red_data = pd.read_csv("Red.csv", encoding='latin-1')

In [25]:
vivino_red_data.shape

(8666, 8)

In [26]:
vivino_red_data

,ï»¿Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Pomerol 2011,France,Pomerol,ChÃ¢teau La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,ChÃ¢teau Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016
...,...,...,...,...,...,...,...,...
8661,6th Sense Syrah 2016,United States,Lodi,Michael David Winery,3.8,994,16.47,2016
8662,Botrosecco Maremma Toscana 2016,Italy,Maremma Toscana,Le Mortelle,4.0,995,20.09,2016
8663,Haut-MÃ©doc 2010,France,Haut-MÃ©doc,ChÃ¢teau Cambon La Pelouse,3.7,996,23.95,2010
8664,Shiraz 2019,Australia,South Eastern Australia,Yellow Tail,3.5,998,6.21,2019


In [27]:
vivino_red_data['Region'].value_counts()

Region
Rioja                                   325
Toscana                                 249
Stellenbosch                            232
Saint-Ãmilion Grand Cru                207
Barolo                                  175
                                       ... 
Sachsen                                   1
Coteaux Varois en Provence                1
Orvietano Rosso                           1
Mercurey 1er Cru 'Le Clos L'EvÃªque'      1
Mount Veeder                              1
Name: count, Length: 624, dtype: int64

In [28]:
vivino_red_data.rename(columns={'ï»¿Name': 'Name'}, inplace=True)

In [29]:
vivino_red_data

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Pomerol 2011,France,Pomerol,ChÃ¢teau La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,ChÃ¢teau Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016
...,...,...,...,...,...,...,...,...
8661,6th Sense Syrah 2016,United States,Lodi,Michael David Winery,3.8,994,16.47,2016
8662,Botrosecco Maremma Toscana 2016,Italy,Maremma Toscana,Le Mortelle,4.0,995,20.09,2016
8663,Haut-MÃ©doc 2010,France,Haut-MÃ©doc,ChÃ¢teau Cambon La Pelouse,3.7,996,23.95,2010
8664,Shiraz 2019,Australia,South Eastern Australia,Yellow Tail,3.5,998,6.21,2019


In [30]:
vivino_white_data = pd.read_csv("White.csv", encoding='latin-1')

In [31]:
vivino_white_data.shape

(3764, 8)

In [32]:
vivino_white_data

,ï»¿Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Vermentino 2017,Italy,Toscana,Famiglia Castellani,3.8,25,5.65,2017
1,Ronco Broilo 2010,Italy,Colli Orientali del Friuli,Conte d'Attimis Maniago,4.3,25,44.90,2010
2,Weisser Schiefer s 2017,Austria,SÃ¼dburgenland,Weinbau Uwe Schiefer,4.2,25,33.25,2017
3,Chardonnay 2018,Germany,Rheinhessen,KrÃ¤mer - Straight,3.9,25,8.99,2018
4,Maganza Zibibbo 2018,Italy,Terre Siciliane,Luna Gaia,3.9,25,8.60,2018
...,...,...,...,...,...,...,...,...
3759,Sauvignon Blanc 2019,New Zealand,Marlborough,Oyster Bay,4.0,4423,10.66,2019
3760,Vinho Verde Sweet N.V.,Portugal,Vinho Verde,Casal Garcia,4.0,4609,5.05,N.V.
3761,Sauvignon Blanc 2018,New Zealand,Marlborough,Kim Crawford,3.9,5105,14.90,2018
3762,Sauvignon Blanc 2019,New Zealand,Marlborough,Hans Greyl,4.2,5817,7.75,2019


In [33]:
vivino_white_data.rename(columns={'ï»¿Name': 'Name'}, inplace=True)
vivino_white_data

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Vermentino 2017,Italy,Toscana,Famiglia Castellani,3.8,25,5.65,2017
1,Ronco Broilo 2010,Italy,Colli Orientali del Friuli,Conte d'Attimis Maniago,4.3,25,44.90,2010
2,Weisser Schiefer s 2017,Austria,SÃ¼dburgenland,Weinbau Uwe Schiefer,4.2,25,33.25,2017
3,Chardonnay 2018,Germany,Rheinhessen,KrÃ¤mer - Straight,3.9,25,8.99,2018
4,Maganza Zibibbo 2018,Italy,Terre Siciliane,Luna Gaia,3.9,25,8.60,2018
...,...,...,...,...,...,...,...,...
3759,Sauvignon Blanc 2019,New Zealand,Marlborough,Oyster Bay,4.0,4423,10.66,2019
3760,Vinho Verde Sweet N.V.,Portugal,Vinho Verde,Casal Garcia,4.0,4609,5.05,N.V.
3761,Sauvignon Blanc 2018,New Zealand,Marlborough,Kim Crawford,3.9,5105,14.90,2018
3762,Sauvignon Blanc 2019,New Zealand,Marlborough,Hans Greyl,4.2,5817,7.75,2019


In [34]:
vivino_data = pd.concat([vivino_red_data, vivino_white_data], axis=0)

In [35]:
vivino_data.shape

(12430, 8)

In [36]:
vivino_data

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Pomerol 2011,France,Pomerol,ChÃ¢teau La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,ChÃ¢teau Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016
...,...,...,...,...,...,...,...,...
3759,Sauvignon Blanc 2019,New Zealand,Marlborough,Oyster Bay,4.0,4423,10.66,2019
3760,Vinho Verde Sweet N.V.,Portugal,Vinho Verde,Casal Garcia,4.0,4609,5.05,N.V.
3761,Sauvignon Blanc 2018,New Zealand,Marlborough,Kim Crawford,3.9,5105,14.90,2018
3762,Sauvignon Blanc 2019,New Zealand,Marlborough,Hans Greyl,4.2,5817,7.75,2019


In [37]:
vivino_data['Name'] = vivino_data['Name'].str.extract(r'([^\d]+)')

In [38]:
vivino_data.drop(columns=['NumberOfRatings'], inplace = True)

In [45]:
vivino_data.rename(columns= lambda x: x.lower(), inplace=True)
vivino_data.rename(columns= lambda x: x.replace(" ", "_"), inplace=True)
vivino_data

,name,country,region,winery,rating,price,year
0,Pomerol,France,Pomerol,ChÃ¢teau La Providence,4.2,95.00,2011
1,Lirac,France,Lirac,ChÃ¢teau Mont-Redon,4.3,15.50,2017
2,Erta e China Rosso di Toscana,Italy,Toscana,Renzo Masi,3.9,7.45,2015
3,Bardolino,Italy,Bardolino,Cavalchina,3.5,8.72,2019
4,Ried Scheibner Pinot Noir,Austria,Carnuntum,Markowitsch,3.9,29.15,2016
...,...,...,...,...,...,...,...
3759,Sauvignon Blanc,New Zealand,Marlborough,Oyster Bay,4.0,10.66,2019
3760,Vinho Verde Sweet N.V.,Portugal,Vinho Verde,Casal Garcia,4.0,5.05,N.V.
3761,Sauvignon Blanc,New Zealand,Marlborough,Kim Crawford,3.9,14.90,2018
3762,Sauvignon Blanc,New Zealand,Marlborough,Hans Greyl,4.2,7.75,2019


In [46]:
vivino_data.dtypes

name        object
country     object
region      object
winery      object
rating     float64
price      float64
year        object
dtype: object

#### Categorical Data: 
Name, Country, Region, Winery, Year

#### Numerical Data: 
Rating, Price

In [41]:
vivino_data.isna().sum() #NO NAN VALUES

name       0
country    0
region     0
winery     0
rating     0
price      0
year       0
dtype: int64

In [47]:
vivino_data['name'].nunique() #ALL UNIQUE VALUES

6835

In [48]:
vivino_data['country'].nunique()

32

In [49]:
vivino_data['region'].nunique()

804

In [50]:
vivino_data['rating'].value_counts()

rating
3.8    1809
3.9    1616
3.7    1476
4.0    1298
4.1    1226
3.6    1165
4.2     940
3.5     723
4.3     583
3.4     434
4.4     342
3.3     258
4.5     180
3.2     132
4.6     111
3.1      45
3.0      32
4.7      29
2.9       8
2.8       8
4.8       7
2.6       2
2.7       2
2.5       2
4.9       1
2.2       1
Name: count, dtype: int64

In [51]:
#THE TOP 10 VARIETIES WITH THE HIGHEST value_count Rating 3.8

top_rating = vivino_data['rating'].value_counts().idxmax()

variety_top_value_rating = vivino_data[vivino_data['rating'] == top_rating]['name']#.iloc[0]

variety_top_value_rating.value_counts().head(10)

name
Cabernet Sauvignon     35
Chardonnay             21
Riesling Trocken       20
Haut-MÃ©doc            18
Sauvignon Blanc        17
Chianti Classico       15
Pinot Noir             13
Chablis                12
Riesling               12
Pinot Grigio           11
Name: count, dtype: int64

In [52]:
len(variety_top_value_rating)

1809

In [53]:
vivino_data['name'].value_counts()

name
Chardonnay                                    175
Cabernet Sauvignon                            175
Sauvignon Blanc                               153
Pomerol                                       104
Saint-Julien (Grand Cru ClassÃ©)               87
                                             ... 
Hugonis                                         1
Finca Biniagual VerÃ¡n                          1
Paz Malbec                                      1
One Bottle of my Cabernet Sauvignon Beats       1
Vinho Verde Branco N.V.                         1
Name: count, Length: 6835, dtype: int64

In [54]:
# Sort the countries based on the count of entries in descending order
vivino_sorted_countries = vivino_data['country'].value_counts().sort_values(ascending=False)

# Select the top 10 countries
top_10_countries_vivino = vivino_sorted_countries.head(10)

print(top_10_countries_vivino)

country
Italy            3442
France           2909
Spain            1398
Germany          1136
South Africa      796
United States     509
Austria           463
Chile             425
Portugal          319
Australia         313
Name: count, dtype: int64


The top country of the original data is number 6 in this dataset

In [55]:
#top province within the US
top_region_us = vivino_data['region'][vivino_data['country'] == 'United States'].value_counts().sort_values(ascending=False).head(50)
print(top_region_us)

region
California                          133
Napa Valley                          87
Columbia Valley                      43
Russian River Valley                 19
Livermore Valley                     17
Sonoma Coast                         16
Lodi                                 13
Sonoma County                        12
Central Coast                        12
Los Carneros                         11
Monterey                             11
Washington                           10
Dundee Hills                          8
Oakville                              8
Dry Creek Valley                      8
St. Helena                            8
North Coast                           8
Sta. Rita Hills                       7
Willamette Valley                     7
Walla Walla Valley                    6
Mendocino County                      6
Paso Robles                           5
Horse Heaven Hills                    5
Alexander Valley                      5
Long Island                      

Most of these regions are within California; reason why region has so many unique values.